In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import keras
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
import numpy as np
import glob
import os
import cv2 as cv
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

## Preprocessing Data

In [2]:
imagestore=[]
video_source_path= "data_set_final"
fps=2 # skips 1 fps
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)
def remove_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img = cv.imread(image_path)
	gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
	gray = cv.resize(gray,(227,227),interpolation=cv.INTER_LINEAR_EXACT)
	imagestore.append(gray)

In [3]:
videos = os.listdir(video_source_path)
create_dir(video_source_path+'/frames')
remove_images(video_source_path+'/frames')
framepath = video_source_path+'/frames'
for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)
			
imagestore=np.array(imagestore)
print(imagestore.shape)
a,b,c=imagestore.shape
imagestore.resize(b,c,a)
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
imagestore=np.clip(imagestore,0,1)
np.save('training.npy',imagestore)
os.system('rm -r {}'.format(framepath))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data_set_final'

In [ ]:
# loading training data which is in .npy file
X_train=np.load('training.npy')
print(X_train.shape)
frames=X_train.shape[2]
frames

FileNotFoundError: [Errno 2] No such file or directory: 'training.npy'

In [ ]:
frames=frames-frames%10

print(frames)

X_train=X_train[:,:,:frames]
print(X_train.shape)
# reshaping Xtrain to 10 X 1 X 227 X 227

X_train=X_train.reshape(-1,227,227,1)
print(X_train.shape)

# X_train=np.expand_dims(X_train,axis=4)
print(X_train.shape)

Y_train=X_train.copy()


NameError: name 'frames' is not defined

In [ ]:
print(X_train[:,:,:1])

NameError: name 'X_train' is not defined

In [ ]:
print(X_train.shape)

NameError: name 'X_train' is not defined

In [ ]:
batch_size = 4

### Spatio Temporal Model

In [ ]:
def loadModel():


    STModel = Sequential()

    STModel.add(Conv3D(filters=128, kernel_size=(11, 11, 1), strides=(4, 4, 1), padding='valid', input_shape=(227, 227, 1,1), activation='relu'))
    STModel.add(Conv3D(filters=64, kernel_size=(5, 5, 1), strides=(2, 2, 1), padding='valid', activation='relu'))

    STModel.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', dropout=0.4, recurrent_dropout=0.3, return_sequences=True))
    STModel.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, return_sequences=True, padding='same', dropout=0.5))

    STModel.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='relu'))
    STModel.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='relu'))


    STModel.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

    return STModel


In [ ]:
epochs = 5

In [ ]:
import tensorflow as tf

In [ ]:
model=loadModel()

save = ModelCheckpoint("behaviourtest_3.h5",
									monitor="mse", save_best_only=False)



stopping_point = EarlyStopping(monitor='loss', patience=3)


print('Model has been loaded')



model.fit(X_train,Y_train,
				batch_size=batch_size,
				epochs=2,
				callbacks = [save,stopping_point]
				)



2022-11-22 02:18:23.899927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 02:18:23.925268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 02:18:23.925665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 02:18:23.927477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Model has been loaded


NameError: name 'X_train' is not defined

In [ ]:
from keras.models import load_model
import numpy as np
def mean_squared(x1, x2):
    difference = x1 - x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_diff = difference ** 2
    Sum = sq_diff.sum()
    dist = np.sqrt(Sum)
    mean_dist = dist / n_samples
    return mean_dist

In [ ]:
threshold=1.16e-4

## Return True if there are abnormal events in Video.

In [ ]:
def predict_anomly(weights_path,path_to_npy_processed_video_file):
    model=load_model(weights_path)

    X_test=np.load(path_to_npy_processed_video_file)
    print(X_test.shape)
    frames=X_test.shape[2]

    Anomality  = False

    frames=frames-frames%10

    X_test=X_test[:,:,:frames]
    X_test=X_test.reshape(-1,227,227,1)
    # X_test=np.expand_dims(X_test,axis=4)
    a_frames = []
    for number,bunch in enumerate(X_test):
        n_bunch=np.expand_dims(bunch,axis=0)
        reconstructed_bunch=model.predict(n_bunch)
        loss=mean_squared(n_bunch,reconstructed_bunch)
        print(loss,end="\r")
        writer.add_scalar("anomality",loss,number)
        writer.flush()
        if loss>threshold:
            print("Anomalous frames @ {}".format(number),end="\r")
            a_frames.append(number)
            Anomality = True
        else:
            continue

    if len(a_frames) > 0:
        
        return Anomality,a_frames
    else:
        a = []
        return Anomality,a

In [ ]:
! nvtop

/bin/bash: line 1: nvtop: command not found


In [ ]:
print(predict_anomly("behaviour_2.h5","f4.npy"))

OSError: No file or directory found at behaviour_2.h5